In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import image
from PIL import Image
import glob
from google.colab import drive
from keras.models import load_model

drive.mount('/content/gdrive')

# Specify the input shape to the first convolutional layer
input_shape = (120, 160, 3)
nClasses = 2
def createModel():
    model = Sequential()
    # a convolution layer of 32 features of size 3x3 with relu activation and zero padding
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    # a convolution layer of 32 features of size 3x3 with relu activation
    model.add(Conv2D(32, (3, 3), activation='relu'))
    # a batch normalization layer
    model.add(BatchNormalization())
    # maxpooling layer of filter size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # a convolution layer of 64 features of size 3x3 with relu activation and zero padding
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    # a convolution layer of 64 features of size 3x3 with relu activation
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # a batch normalization layer
    model.add(BatchNormalization())
    # maxpooling layer of filter size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # a convolution layer of 64 features of size 3x3 with relu activation and zero padding
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    # a convolution layer of 64 features of size 3x3 with relu activation
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # a batch normalization layer
    model.add(BatchNormalization())
    # maxpooling layer of filter size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # a dropout layer of 50%
    model.add(Dropout(0.5))
    
    # flatten the output of the previous layer
    model.add(Flatten())
    # add a dense layer that outputs 512 units and apply relu activation
    model.add(Dense(512, activation='relu'))
    # a dropout layer of 50%
    model.add(Dropout(0.5))
    # add a dense layer with a softmax activation to classify the images
    model.add(Dense(nClasses, activation='softmax'))
     
    return model
  
model2 = createModel()
model2.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Loading the data into a generator
- SPE is argmument used later for  steps per epoch for training and validation

In [2]:
# !ls "/content/gdrive/My Drive/Colab Notebooks/content/binaryD/Data"

# odata = []
# print(len(sorted(glob.glob('/content/gdrive/My Drive/Colab Notebooks/content/binaryD/Data/train/positive/*.png'))))
# for filename in sorted(glob.glob('/content/gdrive/My Drive/Colab Notebooks/content/binaryD/Data/train/*.png')): 
#     im=Image.open(filename)
#     odata.append(im)
    
# print("odatalen: ",len(odata))

trainGen = image.ImageDataGenerator(featurewise_center=False, samplewise_center=True, featurewise_std_normalization=False, samplewise_std_normalization=True, fill_mode='nearest',validation_split = 0.2)
testGen = image.ImageDataGenerator(featurewise_center=False, samplewise_center=True, featurewise_std_normalization=False, samplewise_std_normalization=True, channel_shift_range=0.0, fill_mode='nearest')
mainPath = '/content/gdrive/My Drive/Colab Notebooks/content/binaryD/'
++6
train_generator = trainGen.flow_from_directory(mainPath+'Data/train', target_size=(120, 160), batch_size=32, class_mode='binary',subset = "training")
validation_generator = trainGen.flow_from_directory(mainPath+'Data/train', target_size=(120, 160), batch_size=32, class_mode='binary',subset = "validation")
test_generator = testGen.flow_from_directory(mainPath+'Data/test', target_size=(120, 160), batch_size=32, class_mode='binary')
SPE = 3872/32
print(train_generator[0][0].shape)


Found 3098 images belonging to 2 classes.
Found 774 images belonging to 2 classes.
Found 860 images belonging to 2 classes.
(32, 120, 160, 3)


### Fitting the model
- use keras callbacks for saving the best model and early stopping
- steps per epoch = num train / batchsize
- steps per epoch = num validation / batchsize

both arguments control th data to be transfered using the generator. since each batch has a size of batch size, using the numbers above we make sure that the generator produce the whole training and validation data, In other words it produces steps per epoch  batches of training data. The test and validation data generator behaves in the same manner.


### Complete training the provided model

In [3]:
train_generator.reset()
model2 = load_model(mainPath+"Weights/weights.hdf5")
saveBest2 = keras.callbacks.ModelCheckpoint(mainPath+"model2SampleNorm.hdf5", monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
earlyStopping2 = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=15, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

model2.fit_generator(train_generator, epochs=30, verbose = 1,validation_data = validation_generator,steps_per_epoch = SPE*0.8,validation_steps = SPE*0.2,callbacks = [saveBest2,earlyStopping2])


Epoch 1/30
97/96 [==============================] - 573s 6s/step - loss: 0.4278 - acc: 0.8714 - val_loss: 0.6333 - val_acc: 0.8540
Epoch 2/30
97/96 [==============================] - 570s 6s/step - loss: 0.3726 - acc: 0.8889 - val_loss: 0.7857 - val_acc: 0.8501
Epoch 3/30
97/96 [==============================] - 566s 6s/step - loss: 0.3592 - acc: 0.9022 - val_loss: 0.8933 - val_acc: 0.8437
Epoch 4/30
97/96 [==============================] - 568s 6s/step - loss: 0.3326 - acc: 0.9123 - val_loss: 0.7618 - val_acc: 0.8760
Epoch 5/30
97/96 [==============================] - 570s 6s/step - loss: 0.3304 - acc: 0.9178 - val_loss: 1.0282 - val_acc: 0.8708
Epoch 6/30
97/96 [==============================] - 567s 6s/step - loss: 0.2617 - acc: 0.9283 - val_loss: 0.5203 - val_acc: 0.9096
Epoch 7/30
97/96 [==============================] - 569s 6s/step - loss: 0.2379 - acc: 0.9388 - val_loss: 0.9508 - val_acc: 0.8514
Epoch 8/30
97/96 [==============================] - 567s 6s/step - loss: 0.2346 - a

### Model evaluation
- validation and testing results
- it should be noted that the numbers below change when performing multiple runs 

In [20]:
# test_generator.reset()
TPE=860/32
model2 = load_model(mainPath+"model2SampleNorm.hdf5")
print(model2.metrics_names," ===validation===>",model2.evaluate_generator(generator = validation_generator,steps =SPE*0.2 ))

print(model2.metrics_names," ===test===>",model2.evaluate_generator(generator = test_generator,steps =TPE ))
# model2.save(mainPath+"model2.h5")
# model2 = load_model(mainPath+"model2.h5")

['loss', 'acc']  ===validation===> [0.6089755097889636, 0.9211886304909561]
['loss', 'acc']  ===test===> [0.30915737449841524, 0.9593023253041645]
